# 3D structure analysis and visualization using BioPython
- Sequence analysis is very important, but ultimately, biological function is determined by structure. Here is the FASTA sequence of keap1 kelch domain:

```
MGHAPKVGRLIYTAGGYFRQSLSYLEAYNPSDGTWLDLADLQVPRSGLAGCVVGGLLYAVGGRNNSPDGNTDSSALDCYNPMTNQWSPCAPMSVPRNRIGVGVIDGHIYAVGGS
HGCIHHNSVERYEPERDEWHLVAPMLTRRIGVGVAVLNRLLYAVGGFDGTNRLNSAECYYPERNEWRMITAMNTIRSGAGVCVLHNCIYAAGGYDGQDQLNSVERYDVETETWT
FVAPMKHRRSALGITVHQGRIYVLGGYDGHTFLDSVECYDPDTDTWSEVTRMTSGRSGVGVAVTLEHHHHHH
```

- This protein is critically important for cellular defense against oxidative stress, making it central to antioxidant regulation. Activators of Nrf2 (e.g., certain phytochemicals like lignans) are being explored as potential drugs. You can access our published article, [The promising antioxidant effects of lignans: Nrf2 activation comes into view](https://link.springer.com/article/10.1007/s00210-024-03102-x)

- This big string of text doesn't tell us much about how it can actually do this. Structural biologists are able to determine what that sequence looks like when it takes its natural shape in the cell. Here is that sequence's [structure](https://www.rcsb.org/3d-view/4L7B) in 3D.
Clearly there is a lot more information coded in the sequence that we need to consider. Today I'll walk you through extracting some useful information from this particular protein using BioPython's `PDB` module.

### What is a structure?

- Biomolecules such as proteins, RNA, and DNA are made up of atoms. Each atom occupies a point in 3 dimensions: X, Y, and Z.
Structural biologists isolate molecules in the lab, and using tools such as X-ray crystallography and obtain the X, Y, and Z coordinates of all the atoms in the molecule.
Other notable structure solving technologies are: cryo-electron microscopy and Nuclear Magnetic Resonance (NMR).

- We don't have to worry about how they did it, but for many years now, solved structures (3D positions of biomolecules atoms) have been deposited in the [RCSB PDB](https://www.rcsb.org/)



### Downloading a Structure File from the PDB database

- Every entry in the database has a unique ID code. The keap1 kelch domain we are interested in ID is: 4L7B. The main file format for sequences is `FASTA`, for structures it is `PDB` or `mmCIF`. BioPython.PDB has parsers for both formats.
- [Documentation](http://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ) is the main BioPython.PDB reference page. We can automatically download a structure from the database using the `PDBList` object's method `retrieve_pdb_file`.

### Required packages are : 
- `os`, `Biopython`, `nglview`, and `numpy`.

In [ ]:
import Bio
import os
import nglview as nv
import numpy as np

In [ ]:
from Bio.PDB import *
pdbl = PDBList()
pdb_code = input("Enter your PDB ID : ").lower()
pdbl.retrieve_pdb_file(pdb_code)

- Don't worry about warrning 
- It tells us that the PDB format is deprecated and that the new default is mmCIF.
- Now we have a folder named as `l7` and containing `4l7b.cif` in our working directory.

In [ ]:
os.listdir()

In [ ]:
dir_path = input("Enter the path of the created directory : ")
os.listdir(dir_path)

### Parsing Structure files
- The first argument in `get_structure` method is an optional name for the macromolecule, and the second argument is the path to the structure file.

In [ ]:
parser = MMCIFParser()
file_path = input("Enter the path of the cif file : ")
file_name = input("Enter the file name : ")
structure = parser.get_structure(file_name, file_path)

- Now we have extracted all the structural information from the file. Let's take a peek at the attributes.

In [ ]:
def cleandir(obj):
    print(", ".join([a for a in dir(obj) if not a.startswith("_")]))
cleandir(structure)

- Structure objects are organized in a specific hierarchy of objects. We're just going to focus on the core elements which are: Model --> Chain --> Residue --> Atom.
- Each structure file can contain multiple "models" of the same molecule. Each model contains several "chains" or strands of protein/RNA/DNA/.
- Each "chain" is made up of residues, or amino acids/DNA bases/RNA bases.
- Each residue is made up of Atoms.

### Visualizing structures in jupyter Notebooks using nglview

- To do more advanced manipulations, it is better to use standalone tools such as [Chimera](https://www.cgl.ucsf.edu/chimera/) and [pyMOL](https://pymol.org/2/).

### Let's view our keap1 kelch domain bounded to the co-crystalized `1VV`.

In [ ]:
view = nv.show_biopython(structure)
view.clear_representations()
#view as ball and stick (atom and bond)
view.add_ball_and_stick()
view

- As expected, we see a bunch of atoms in 3D space. This can often be hard to look at so we usually visualize this with what's known as a "ribbon" representation.

In [ ]:
view.clear_representations()
#add ribbons
view.add_cartoon('protein')
#add ball and stick for non-rotien
view.add_ball_and_stick('not protein')
view

- Individual atoms are not shown and instead, each chain is visualized as a continuous ribbon. We can see here that we have 2 different chains, 1 co-crystalized ligand and other co-factors like `ACT` and `NA`.
- Let's see how we can get a handle on the different components.
- How many models are there in this structure?

In [ ]:
for model in structure:
    print(f"model {model}")

- It looks like there is only one protein so we only have a single model.
- But we have several chains, as we saw above.

In [ ]:
model = structure[0] #since we only have one model
for chain in model:
    print(f"chain {chain}, Chain ID: {chain.id}")

- Ok as we guessed, we have 2 chains: A and B. We can go one step further and get all the residues in any chain. 
- We can access individual chains like keys in a dictionary from a model.

In [ ]:
chain_A = model['A']
for res in chain_A:
    print(f"Residue name: {res.resname}, number: {res.id[1]}")

- Now we see that each chain is made up of `Residue`. 
- Finally, each residue should be made up of atoms. 
- Let's get residue 338 from Chain A then show it's atoms as an example .

In [ ]:
res = chain_A[338]
print(f"atoms of {res} are : ")

for atom in res:
    print(f"{atom.name}")

- Putting it all together, we can print every atom in every residue in the specified chain in our model.

In [ ]:
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                print(atom)

- Let's say if we want to design a drug that binds to the keap1 kelch domain and activate it. Given its central role in controlling oxidative stress, modulating this pathway has potential in treating diseases driven by oxidative damage, such as in neurodegenerative diseases and cancer.
- In order to do this, we need to know which residues co-crystalized `1VV` binds so we can design a chemical that targets these kinds of residues.

## mmCIF annotations
mmCIF files are like dictionaries with {key:value}. When we parsed the file above, we really only got the structure info, but there is a lot more information that we can access by parsing a little differently.


MMCIF files have a ton of information and they look like this:

```
loop_
_pdbx_nonpoly_scheme.asym_id 
_pdbx_nonpoly_scheme.entity_id 
_pdbx_nonpoly_scheme.mon_id 
_pdbx_nonpoly_scheme.ndb_seq_num 
_pdbx_nonpoly_scheme.pdb_seq_num 
_pdbx_nonpoly_scheme.auth_seq_num 
_pdbx_nonpoly_scheme.pdb_mon_id 
_pdbx_nonpoly_scheme.auth_mon_id 
_pdbx_nonpoly_scheme.pdb_strand_id 
_pdbx_nonpoly_scheme.pdb_ins_code 
C 2 ACT 1   701 1   ACT ACT A . 
D 2 ACT 1   702 3   ACT ACT A . 
E 2 ACT 1   703 4   ACT ACT A . 
F 2 ACT 1   704 1   ACT ACT A . 
G 3 1VV 1   701 1   1VV UNL B . 
H 4 NA  1   702 1   NA  NA  B . 
I 2 ACT 1   703 2   ACT ACT B .  
```
- This happens to be an entry for "non-polymer" entities ligands/chemicals. There are 10 `_pdbx_nonpoly_scheme..` headers. Each corresponds to a label for the columns in the entries below.
- So `_pdbx_nonpoly_scheme.mon_id` corresponds to the third column which is the `id` of the ligand.
- The `loop_` means, when parsing, loop over all the rows for each column label to make a list.
- For example: `_pdbx_nonpoly_scheme.asym_id ` after looping would map to: `['C', 'D', 'E', 'F', 'G', 'H', 'I']`
- We're interested in `1VV` which is the co-crystalized ligand, but we see that there are others, lets parse this information into a dictionary.

In [ ]:
struc_dict = MMCIF2Dict.MMCIF2Dict('l7/4l7b.cif')
print(struc_dict.keys())

As you can see we have tons of keys.
Binding sites are often annotated by the researches in their mmCIF files.

```
loop_
_struct_site.id 
_struct_site.pdbx_evidence_code 
_struct_site.pdbx_auth_asym_id 
_struct_site.pdbx_auth_comp_id 
_struct_site.pdbx_auth_seq_id 
_struct_site.pdbx_auth_ins_code 
_struct_site.pdbx_num_residues 
_struct_site.details 
AC1 Software A ACT 701 ? 7 'BINDING SITE FOR RESIDUE ACT A 701' 
AC2 Software A ACT 702 ? 3 'BINDING SITE FOR RESIDUE ACT A 702' 
AC3 Software A ACT 703 ? 5 'BINDING SITE FOR RESIDUE ACT A 703' 
AC4 Software A ACT 704 ? 7 'BINDING SITE FOR RESIDUE ACT A 704' 
AC5 Software B 1VV 701 ? 8 'BINDING SITE FOR RESIDUE 1VV B 701' 
AC6 Software B NA  702 ? 3 'BINDING SITE FOR RESIDUE NA B 702'  
AC7 Software B ACT 703 ? 6 'BINDING SITE FOR RESIDUE ACT B 703'  
```

In [ ]:
struc_dict['_struct_site.details']

- So the binding site for `1VV` has ID `AC5`.

- There are other useful entries such as `the title of the publication` that this structure was featured in.

In [ ]:
struc_dict['_citation.title']

- All entry types are documented [here](http://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Index/).
- Let's extract all residues in the `1VV` binding site.

In [ ]:
site_ID = struc_dict['_struct_site_gen.site_id']
site_chain = struc_dict['_struct_site_gen.auth_asym_id']
site_resnum = struc_dict['_struct_site_gen.auth_seq_id']
site_resname = struc_dict['_struct_site_gen.label_comp_id']

cif_binding_residues = []
for bind_id, ch, num, name in zip(site_ID, site_chain, site_resnum, site_resname):
    if bind_id == "AC5":
        print(bind_id, ch, num, name)
        try:
            cif_binding_residues.append(structure[0][ch][int(num)])
        except:
            continue
    else:
        continue

- Let's say we don't trust this annotation. We can compute our own ligand binding site and compare.
- NOTE: Obviously we could have done this directly with the `structure` object we obtained earlier.

### Step 1: Find the 1VV residue
- We can get all the residues of a model using the `model.get_residues()` method.

In [ ]:
#finding 1VV residue
residues = None
for res in structure[0].get_residues():
    if res.resname == "1VV":
        residues = res
        break
print(residues)

### Step 2: find all other residues within a certain distance cutoff
- Let's set a distance cutoff of 10 Angstroms. We will count any residue whose $\alpha$-carbon is within 10 Angstrom of any atom in `1VV`.
- BioPython gives us a vector of X, Y, Z coordinates for every atom in the residue. Let's get the X, Y, Z coordinates of the alpha carbon of `1VV` residue then take the average.
- For this, we use the `coord` attribute.

In [ ]:
res_1_CA = structure[0]['B'][334]['CA']
res_2_CA = structure[0]['B'][414]['CA']
res_3_CA = structure[0]['B'][415]['CA']
res_4_CA = structure[0]['B'][509]['CA']
res_5_CA = structure[0]['B'][556]['CA']
res_6_CA = structure[0]['B'][572]['CA']
res_7_CA = structure[0]['B'][602]['CA']

coords = []

coords.append(res_1_CA.coord)
coords.append(res_2_CA.coord)
coords.append(res_3_CA.coord)
coords.append(res_4_CA.coord)
coords.append(res_5_CA.coord)
coords.append(res_6_CA.coord)
coords.append(res_7_CA.coord)

average_coord = np.mean(coords, axis=0)

# Print each coordinate and the average
for i, coord in enumerate(coords, start=1):
    print(f"Residue {i} CA coordinate: {coord}")
print("-" * 50)
print(f"Average coordinate: {average_coord}")

- Now we can use this to compute distances!

In [ ]:
cutoff = 10
binding_residues = []

for res in structure[0].get_residues():
    # Skip the 1VV residue
    if res == residues:
        continue
    # Non-amino acid residues are tagged with an 'H' in their id tuple
    # We want to skip these
    elif res.id[0].startswith("H"):
        continue
    # Check if the residue contains a "CA" atom
    elif "CA" not in res:
        continue
    else:
        alpha_carbon = res["CA"]
        distances = []
        for atom in residues:
            # Subtract the two position vectors
            diff_vector = alpha_carbon.coord - atom.coord
            # Calculate the Euclidean distance
            distances.append(np.linalg.norm(diff_vector))
        # Check if the minimum distance falls within the cutoff
        if min(distances) < cutoff:
            binding_residues.append(res)
print(binding_residues)

### Visualize the structure

In [ ]:
view = nv.show_biopython(structure)

# Iterate through residues and assign colors
for res in structure[0].get_residues():
    # Default color (blue) for non-binding residues
    color = "blue"
    if res in binding_residues:  # Check if residue is a binding residue
        color = "green"  # Color for binding residues
    # Add the residue with the color to the representation
    view.add_representation(
        "ball+stick",
        selection=f"{res.get_full_id()[3][1]}:{res.get_full_id()[2]}",
        color=color,
    )

# Display the structure
view

- Now let's compare this to our list of residues obtained from the mmCIF annotations.

In [ ]:
view = nv.show_biopython(structure)

# Use the `add_representation` method to color residues
for res in structure[0].get_residues():
    # Assign colors based on whether the residue is in cif_binding_residues
    color = "green" if res in cif_binding_residues else "blue"
    residue_id = res.get_id()[1]  # Get the residue number
    chain_id = res.get_full_id()[2]  # Get the chain ID
    # Add representation with specific color
    view.add_representation(
        "spacefill",
        selection=f"{residue_id}:{chain_id}",
        color=color,
    )

# Display the structure
view